In [1]:
import requests


res = requests.get('http://35.222.204.206:5000/records')
data = res.json()

data_set = []

for record in data:
    
    inputs={}
    output={}
    
    transcript_result = record["brands_audio"]
    
    if len(transcript_result["gemini_results"]):
        inputs["gemini_results"] =[(k,v) for k,v in  transcript_result["gemini_results"].items()]
    else:
        inputs["gemini_results"] = []
    
    vi_results = [(k,v) for k,v in record["brands_video_gcp"].items()]
    
    if len(vi_results):
        inputs["vi_results"] = vi_results
    else:
        inputs["vi_results"]=[]
    
    
    ocr_result = record["ocr_text"]
    
    if len(ocr_result):
        inputs["ocr_text"] = [(k["brand"],k["confidence"]) for k in ocr_result]
    else:
        inputs["ocr_text"] = []
    
    
    output["final"]=[k for k in record["final_brands"]]
    
    data_set.append({"inputs":dict(inputs),"output":dict(output)})
    
    
print(data_set)
    

[{'inputs': {'gemini_results': [('Carvana', 1), ('Mustang', 1)], 'vi_results': [('Apple Store', 0.99), ('Google Store', 0.99)], 'ocr_text': [('Carvana', 1)]}, 'output': {'final': ['Carvana']}}, {'inputs': {'gemini_results': [('Breeze', 1), ('Febreze', 1)], 'vi_results': [('Febreze', 0.99), ('Procter & Gamble', 0.99)], 'ocr_text': [('Febreze', 0.9)]}, 'output': {'final': ['Febreze']}}, {'inputs': {'gemini_results': [('Dove', 1)], 'vi_results': [('Unilever', 0.99), ('Nike', 0.99)], 'ocr_text': [('Dove', 1), ('Unilever', 0.8)]}, 'output': {'final': ['Unilever', 'Dove']}}, {'inputs': {'gemini_results': [('Paramount+', 1)], 'vi_results': [('Paramount Network', 0.99)], 'ocr_text': [('Paramount+', 0.9)]}, 'output': {'final': ['Paramount+']}}, {'inputs': {'gemini_results': [('Discover Newport', 1)], 'vi_results': [], 'ocr_text': [('Discover Newport', 0.9)]}, 'output': {'final': ['Discover Newport']}}, {'inputs': {'gemini_results': [('Raymour & Flanigan', 1)], 'vi_results': [('Raymour & Flaniga

Binary Loss:

Binary loss (0 for correct prediction, 1 for incorrect prediction) is a simpler measure that only considers whether the final prediction matches the actual label, not how confident that prediction was.

Gradient Calculation:

python
Copy code
if final_brand != actual_final:
    for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
        if any(brand == actual_final for brand, _ in entry['inputs'][source]):
            weight_gradients[i] -= 1
        if any(brand == final_brand for brand, _ in entry['inputs'][source']):
            weight_gradients[i] += 1
If the predicted brand is incorrect, we update the weight gradients:
We decrease the gradient for sources that had the actual final brand.
We increase the gradient for sources that had the incorrectly predicted brand.
Weight Update:

python
Copy code
weights -= learning_rate * weight_gradients
weights = np.clip(weights, 0, 1)
weights /= np.sum(weights)
We adjust the weights using the calculated gradients and the learning rate.
The weights are clipped to ensure they stay between 0 and 1.
The weights are normalized to sum to 1.

In [2]:
import numpy as np
from collections import defaultdict
import copy

# Example dataset
data = copy.deepcopy(data_set)

# Combining confidence scores with weights
def combine_confidences(inputs, weights):
    combined_scores = defaultdict(float)
    for source, weight in zip(['gemini_results', 'vi_results', 'ocr_text'], weights):
        for brand, score in inputs[source]:
            combined_scores[brand] += weight * score
    return combined_scores

# Normalizing the scores
def normalize_scores(combined_scores):
    total_score = sum(combined_scores.values())
    if total_score == 0:
        return combined_scores
    for brand in combined_scores:
        combined_scores[brand] /= total_score
    return combined_scores

# Computing the final prediction
def get_final_brand(inputs, weights):
    combined_scores = combine_confidences(inputs, weights)
    normalized_scores = normalize_scores(combined_scores)
    final_brand = max(normalized_scores, key=normalized_scores.get)
    final_confidence = normalized_scores[final_brand]
    return final_brand, final_confidence

# Calculating the loss function
def calculate_loss(predicted, actual):
    return 1 if predicted != actual else 0

# Optimizing the weights using gradient descent
def optimize_weights(data_set, learning_rate=0.01, epochs=100):
    weights = np.ones(3) / 3  # Initial weights for gemini_results, vi_results, ocr_text
    for epoch in range(epochs):
        total_loss = 0
        weight_gradients = np.zeros(3)
        for entry in data_set:
            actual_final = entry['output']['final'][0]
            final_brand, _ = get_final_brand(entry['inputs'], weights)
            loss = calculate_loss(final_brand, actual_final)
            total_loss += loss
            if final_brand != actual_final:
                for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                    if any(brand == actual_final for brand, _ in entry['inputs'][source]):
                        weight_gradients[i] -= 1
                    if any(brand == final_brand for brand, _ in entry['inputs'][source]):
                        weight_gradients[i] += 1
        weights -= learning_rate * weight_gradients
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}, Weights: {weights}")
    return weights

# Optimizing weights and testing
final_weights = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence = get_final_brand(entry['inputs'], final_weights)
    print(f"Test Data - Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Test Data - Weights: {final_weights}")

# Print results for training data and calculate total loss
total_loss = 0
for entry in data:
    final_brand, final_confidence = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")


Epoch 1/100, Loss: 7, Weights: [0.30952381 0.30952381 0.38095238]
Epoch 2/100, Loss: 7, Weights: [0.29543246 0.29543246 0.40913508]
Epoch 3/100, Loss: 6, Weights: [0.28105353 0.29125761 0.42768886]
Epoch 4/100, Loss: 6, Weights: [0.26638115 0.28699756 0.44662129]
Epoch 5/100, Loss: 6, Weights: [0.25140934 0.28265057 0.46594009]
Epoch 6/100, Loss: 6, Weights: [0.23613198 0.27821487 0.48565315]
Epoch 7/100, Loss: 6, Weights: [0.22054284 0.27368864 0.50576852]
Epoch 8/100, Loss: 4, Weights: [0.22054284 0.28368864 0.49576852]
Epoch 9/100, Loss: 6, Weights: [0.20463555 0.27927413 0.51609033]
Epoch 10/100, Loss: 4, Weights: [0.20463555 0.28927413 0.50609033]
Epoch 11/100, Loss: 4, Weights: [0.20463555 0.29927413 0.49609033]
Epoch 12/100, Loss: 6, Weights: [0.18840362 0.29517768 0.5164187 ]
Epoch 13/100, Loss: 3, Weights: [0.19840362 0.29517768 0.5064187 ]
Epoch 14/100, Loss: 4, Weights: [0.19840362 0.30517768 0.4964187 ]
Epoch 15/100, Loss: 5, Weights: [0.19224859 0.29099763 0.51675378]
Epoc

Revised Code with Cross-Entropy Loss and Adam Optimizer


In [3]:
import numpy as np
from collections import defaultdict
import copy

data = copy.deepcopy(data_set)
# Combining confidence scores with weights
def combine_confidences(inputs, weights):
    combined_scores = defaultdict(float)
    for source, weight in zip(['gemini_results', 'vi_results', 'ocr_text'], weights):
        for brand, score in inputs[source]:
            combined_scores[brand] += weight * score
    return combined_scores

# Normalizing the scores
def normalize_scores(combined_scores):
    total_score = sum(combined_scores.values())
    if total_score == 0:
        return combined_scores
    for brand in combined_scores:
        combined_scores[brand] /= total_score
    return combined_scores

# Computing the final prediction
def get_final_brand(inputs, weights):
    combined_scores = combine_confidences(inputs, weights)
    normalized_scores = normalize_scores(combined_scores)
    final_brand = max(normalized_scores, key=normalized_scores.get)
    final_confidence = normalized_scores[final_brand]
    return final_brand, final_confidence, normalized_scores

# Calculating cross-entropy loss
def cross_entropy_loss(predicted_probs, actual_label, all_brands):
    epsilon = 1e-15
    predicted_probs = np.array([predicted_probs.get(brand, epsilon) for brand in all_brands])
    predicted_probs = np.clip(predicted_probs, epsilon, 1 - epsilon)
    actual_vector = np.array([1 if brand == actual_label else 0 for brand in all_brands])
    return -np.sum(actual_vector * np.log(predicted_probs))

# Simple binary loss for evaluation
def calculate_loss(predicted, actual):
    return 1 if predicted != actual else 0

# Optimizing the weights using Adam optimizer
def optimize_weights(data_set, learning_rate=0.01, epochs=100):
    weights = np.ones(3) / 3  # Initial weights for gemini_results, vi_results, ocr_text
    m = np.zeros(3)
    v = np.zeros(3)
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    t = 0
    
    all_brands = set()
    for entry in data_set:
        for source in ['gemini_results', 'vi_results', 'ocr_text']:
            all_brands.update([brand for brand, _ in entry['inputs'][source]])
    all_brands = list(all_brands)
    
    for epoch in range(epochs):
        total_loss = 0
        weight_gradients = np.zeros(3)
        for entry in data_set:
            actual_final = entry['output']['final'][0]
            final_brand, _, predicted_probs = get_final_brand(entry['inputs'], weights)
            loss = cross_entropy_loss(predicted_probs, actual_final, all_brands)
            total_loss += loss
            for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                for brand, score in entry['inputs'][source]:
                    gradient = (predicted_probs.get(brand, 0) - (1 if brand == actual_final else 0)) * score
                    weight_gradients[i] += gradient
        
        t += 1
        m = beta1 * m + (1 - beta1) * weight_gradients
        v = beta2 * v + (1 - beta2) * (weight_gradients ** 2)
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        weights -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
        
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}, Weights: {weights}")
    return weights

# Optimizing weights and testing
final_weights = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    print(f"Test Data - Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Test Data - Weights: {final_weights}")

# Print results for training data and calculate total loss
total_loss = 0
for entry in data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")


Epoch 1/100, Loss: 25.01246291782739, Weights: [0.33993399 0.32013201 0.33993399]
Epoch 2/100, Loss: 24.556503349931294, Weights: [0.34645844 0.30707765 0.34646391]
Epoch 3/100, Loss: 24.112574863564728, Weights: [0.35289958 0.29418036 0.35292006]
Epoch 4/100, Loss: 23.68045234482656, Weights: [0.35924994 0.28145075 0.35929931]
Epoch 5/100, Loss: 23.259945014984943, Weights: [0.36550164 0.2688999  0.36559846]
Epoch 6/100, Loss: 22.85089482712831, Weights: [0.37164639 0.25653938 0.37181423]
Epoch 7/100, Loss: 22.45317499149614, Weights: [0.37767549 0.24438127 0.37794324]
Epoch 8/100, Loss: 22.066688581650215, Weights: [0.38357986 0.23243811 0.38398203]
Epoch 9/100, Loss: 21.691367171825096, Weights: [0.38935    0.22072293 0.38992707]
Epoch 10/100, Loss: 21.32716945349052, Weights: [0.39497605 0.20924921 0.39577474]
Epoch 11/100, Loss: 20.974079776712266, Weights: [0.4004478  0.19803086 0.40152134]
Epoch 12/100, Loss: 20.632106559749037, Weights: [0.40575474 0.18708215 0.40716311]
Epoch 

Updated Code with Best Weights Tracking

In [4]:
import numpy as np
from collections import defaultdict

# Example dataset
data = data_set

# Combining confidence scores with weights
def combine_confidences(inputs, weights):
    combined_scores = defaultdict(float)
    for source, weight in zip(['gemini_results', 'vi_results', 'ocr_text'], weights):
        for brand, score in inputs[source]:
            combined_scores[brand] += weight * score
    return combined_scores

# Normalizing the scores
def normalize_scores(combined_scores):
    total_score = sum(combined_scores.values())
    if total_score == 0:
        return combined_scores
    for brand in combined_scores:
        combined_scores[brand] /= total_score
    return combined_scores

# Computing the final prediction
def get_final_brand(inputs, weights):
    combined_scores = combine_confidences(inputs, weights)
    normalized_scores = normalize_scores(combined_scores)
    final_brand = max(normalized_scores, key=normalized_scores.get)
    final_confidence = normalized_scores[final_brand]
    return final_brand, final_confidence, normalized_scores

# Calculating cross-entropy loss
def cross_entropy_loss(predicted_probs, actual_label, all_brands):
    epsilon = 1e-15
    predicted_probs = np.array([predicted_probs.get(brand, epsilon) for brand in all_brands])
    predicted_probs = np.clip(predicted_probs, epsilon, 1 - epsilon)
    actual_vector = np.array([1 if brand == actual_label else 0 for brand in all_brands])
    return -np.sum(actual_vector * np.log(predicted_probs))

# Simple binary loss for evaluation
def calculate_loss(predicted, actual):
    return 1 if predicted != actual else 0

# Optimizing the weights using Adam optimizer
def optimize_weights(data_set, learning_rate=0.01, epochs=1000):
    weights = np.ones(3) / 3  # Initial weights for gemini_results, vi_results, ocr_text
    m = np.zeros(3)
    v = np.zeros(3)
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    t = 0
    
    best_weights = weights.copy()
    min_loss = float('inf')
    
    all_brands = set()
    for entry in data_set:
        for source in ['gemini_results', 'vi_results', 'ocr_text']:
            all_brands.update([brand for brand, _ in entry['inputs'][source]])
    all_brands = list(all_brands)
    
    for epoch in range(epochs):
        total_loss = 0
        weight_gradients = np.zeros(3)
        for entry in data_set:
            actual_final = entry['output']['final'][0]
            final_brand, _, predicted_probs = get_final_brand(entry['inputs'], weights)
            loss = cross_entropy_loss(predicted_probs, actual_final, all_brands)
            total_loss += loss
            for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                for brand, score in entry['inputs'][source]:
                    gradient = (predicted_probs.get(brand, 0) - (1 if brand == actual_final else 0)) * score
                    weight_gradients[i] += gradient
        
        if total_loss < min_loss:
            min_loss = total_loss
            best_weights = weights.copy()
        
        t += 1
        m = beta1 * m + (1 - beta1) * weight_gradients
        v = beta2 * v + (1 - beta2) * (weight_gradients ** 2)
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        weights -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
        
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}, Weights: {weights}")
    return best_weights

# Optimizing weights and testing
final_weights = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    print(f"Test Data - Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Test Data - Weights: {final_weights}")

# Print results for training data and calculate total loss
total_loss = 0
for entry in data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")


Epoch 1/1000, Loss: 25.01246291782739, Weights: [0.33993399 0.32013201 0.33993399]
Epoch 2/1000, Loss: 24.556503349931294, Weights: [0.34645844 0.30707765 0.34646391]
Epoch 3/1000, Loss: 24.112574863564728, Weights: [0.35289958 0.29418036 0.35292006]
Epoch 4/1000, Loss: 23.68045234482656, Weights: [0.35924994 0.28145075 0.35929931]
Epoch 5/1000, Loss: 23.259945014984943, Weights: [0.36550164 0.2688999  0.36559846]
Epoch 6/1000, Loss: 22.85089482712831, Weights: [0.37164639 0.25653938 0.37181423]
Epoch 7/1000, Loss: 22.45317499149614, Weights: [0.37767549 0.24438127 0.37794324]
Epoch 8/1000, Loss: 22.066688581650215, Weights: [0.38357986 0.23243811 0.38398203]
Epoch 9/1000, Loss: 21.691367171825096, Weights: [0.38935    0.22072293 0.38992707]
Epoch 10/1000, Loss: 21.32716945349052, Weights: [0.39497605 0.20924921 0.39577474]
Epoch 11/1000, Loss: 20.974079776712266, Weights: [0.4004478  0.19803086 0.40152134]
Epoch 12/1000, Loss: 20.632106559749037, Weights: [0.40575474 0.18708215 0.4071

Updated Code with Hybrid Loss

In [5]:
import numpy as np
from collections import defaultdict

# Example dataset
data = data_set

# Combining confidence scores with weights
def combine_confidences(inputs, weights):
    combined_scores = defaultdict(float)
    for source, weight in zip(['gemini_results', 'vi_results', 'ocr_text'], weights):
        for brand, score in inputs[source]:
            combined_scores[brand] += weight * score
    return combined_scores

# Normalizing the scores
def normalize_scores(combined_scores):
    total_score = sum(combined_scores.values())
    if total_score == 0:
        return combined_scores
    for brand in combined_scores:
        combined_scores[brand] /= total_score
    return combined_scores

# Computing the final prediction
def get_final_brand(inputs, weights):
    combined_scores = combine_confidences(inputs, weights)
    normalized_scores = normalize_scores(combined_scores)
    final_brand = max(normalized_scores, key=normalized_scores.get)
    final_confidence = normalized_scores[final_brand]
    return final_brand, final_confidence, normalized_scores

# Calculating cross-entropy loss
def cross_entropy_loss(predicted_probs, actual_label, all_brands):
    epsilon = 1e-15
    predicted_probs = np.array([predicted_probs.get(brand, epsilon) for brand in all_brands])
    predicted_probs = np.clip(predicted_probs, epsilon, 1 - epsilon)
    actual_vector = np.array([1 if brand == actual_label else 0 for brand in all_brands])
    return -np.sum(actual_vector * np.log(predicted_probs))

# Simple binary loss for evaluation
def calculate_loss(predicted, actual):
    return 1 if predicted != actual else 0

# Optimizing the weights using Adam optimizer with hybrid loss
def optimize_weights(data_set, learning_rate=0.01, epochs=1000):
    weights = np.ones(3) / 3  # Initial weights for gemini_results, vi_results, ocr_text
    m = np.zeros(3)
    v = np.zeros(3)
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    t = 0
    
    best_weights = weights.copy()
    min_binary_loss = float('inf')
    
    all_brands = set()
    for entry in data_set:
        for source in ['gemini_results', 'vi_results', 'ocr_text']:
            all_brands.update([brand for brand, _ in entry['inputs'][source]])
    all_brands = list(all_brands)
    
    for epoch in range(epochs):
        total_cross_entropy_loss = 0
        total_binary_loss = 0
        weight_gradients = np.zeros(3)
        for entry in data_set:
            actual_final = entry['output']['final'][0]
            final_brand, _, predicted_probs = get_final_brand(entry['inputs'], weights)
            cross_entropy = cross_entropy_loss(predicted_probs, actual_final, all_brands)
            binary_loss = calculate_loss(final_brand, actual_final)
            total_cross_entropy_loss += cross_entropy
            total_binary_loss += binary_loss
            for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                for brand, score in entry['inputs'][source]:
                    gradient = (predicted_probs.get(brand, 0) - (1 if brand == actual_final else 0)) * score
                    weight_gradients[i] += gradient
        
        # Update best weights based on binary loss
        if total_binary_loss < min_binary_loss:
            min_binary_loss = total_binary_loss
            best_weights = weights.copy()
        
        t += 1
        m = beta1 * m + (1 - beta1) * weight_gradients
        v = beta2 * v + (1 - beta2) * (weight_gradients ** 2)
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        weights -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
        
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        
        print(f"Epoch {epoch+1}/{epochs}, Cross-Entropy Loss: {total_cross_entropy_loss}, Binary Loss: {total_binary_loss}, Weights: {weights}")
    
    print(f"Best Weights based on Binary Loss: {best_weights}")
    return best_weights

# Optimizing weights and testing
final_weights = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    print(f"Test Data - Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Test Data - Weights: {final_weights}")

# Print results for training data and calculate total loss
total_loss = 0
for entry in data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")


Epoch 1/1000, Cross-Entropy Loss: 25.01246291782739, Binary Loss: 7, Weights: [0.33993399 0.32013201 0.33993399]
Epoch 2/1000, Cross-Entropy Loss: 24.556503349931294, Binary Loss: 7, Weights: [0.34645844 0.30707765 0.34646391]
Epoch 3/1000, Cross-Entropy Loss: 24.112574863564728, Binary Loss: 6, Weights: [0.35289958 0.29418036 0.35292006]
Epoch 4/1000, Cross-Entropy Loss: 23.68045234482656, Binary Loss: 6, Weights: [0.35924994 0.28145075 0.35929931]
Epoch 5/1000, Cross-Entropy Loss: 23.259945014984943, Binary Loss: 5, Weights: [0.36550164 0.2688999  0.36559846]
Epoch 6/1000, Cross-Entropy Loss: 22.85089482712831, Binary Loss: 5, Weights: [0.37164639 0.25653938 0.37181423]
Epoch 7/1000, Cross-Entropy Loss: 22.45317499149614, Binary Loss: 5, Weights: [0.37767549 0.24438127 0.37794324]
Epoch 8/1000, Cross-Entropy Loss: 22.066688581650215, Binary Loss: 5, Weights: [0.38357986 0.23243811 0.38398203]
Epoch 9/1000, Cross-Entropy Loss: 21.691367171825096, Binary Loss: 5, Weights: [0.38935    0

In [6]:
# Print results for training data and calculate total loss
total_loss = 0
wrong_predictios = []
for entry in data:
    final_brand, final_confidence, _ = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    if(loss):
        wrong_predictios.append(entry)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")

Expected: Carvana, Predicted: Carvana, Confidence: 0.6418508814322089, Loss: 0
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: Febreze, Predicted: Febreze, Confidence: 0.6555720240087504, Loss: 0
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: Unilever, Predicted: Dove, Confidence: 0.6688436420383966, Loss: 1
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: Paramount+, Predicted: Paramount+, Confidence: 0.9601829918282818, Loss: 0
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: Discover Newport, Predicted: Discover Newport, Confidence: 1.0, Loss: 0
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: Raymour & Flanigan, Predicted: Raymour & Flanigan, Confidence: 0.9635559593225195, Loss: 0
Training Data - Weights: [0.46106853 0.0381899  0.50074157]
Expected: SmartLife, Predicted: Nectar, Confidence: 0.16864603682761806, Loss: 1
Training Data - Weights: [0.46106853 0.0381899  0.500741

In [7]:
wrong_predictios

[{'inputs': {'gemini_results': [('Dove', 1)],
   'vi_results': [('Unilever', 0.99), ('Nike', 0.99)],
   'ocr_text': [('Dove', 1), ('Unilever', 0.8)]},
  'output': {'final': ['Unilever', 'Dove']}},
 {'inputs': {'gemini_results': [('Nectar', 1),
    ('Sealy', 1),
    ('Casper', 1),
    ('Serta', 1),
    ('Stearns & Foster', 1),
    ('SmartLife', 1),
    ('King Koil', 1)],
   'vi_results': [('Raymour & Flanigan', 0.99), ('Sealy Corporation', 0.99)],
   'ocr_text': [('Nectar', 0.9),
    ('Sealy', 0.8),
    ('Casper', 0.7),
    ('Serta', 0.6),
    ('Stearns & Foster', 0.5),
    ('SmartLife', 0.4),
    ('King Koil', 0.3)]},
  'output': {'final': ['SmartLife',
    'Casper',
    'Serta',
    'King Koil',
    'Stearns & Foster',
    'Nectar',
    'Sealy']}},
 {'inputs': {'gemini_results': [('Disney', 1),
    ('Hulu', 1),
    ('Google', 1),
    ('FX', 1),
    ('20th Century Studios', 1),
    ('Searchlight Pictures', 1)],
   'vi_results': [('The Walt Disney Company', 0.99), ('Hulu', 0.99)],
   'o